# Visualisation de données sous forme d'arbre

A partir d'un jeu de données composé de plusieurs génomes, on souhaite estimer si ces génomes sont proches au niveau de leurs séquences nucléïques. Pour faire ceci, l'outil Mash peut-etre utilisé.

## Mash

[Mash](https://mash.readthedocs.io/en/latest/index.html) repose sur un algorithme de MinHash pour comparer rapidement des données génomiques et propose d’en calculer une distance. La distance de Mash mesure la dissimilarité entre deux échantillons. Plus la distance calculée est proche de 0, plus les séquences sont similaires. La distance de Mash est corrélée à la valeur de l’identité nucléotidique moyenne (*average nucleotide identity*, ANI), une mesure classique de similarité de séquences. La relation entre l’ANI et la distance de Mash D est décrite par la simple relation : `D ≈ 1 - ANI`.

*Reference :* [Mash: fast genome and metagenome distance estimation using MinHash. Ondov BD, Treangen TJ, Melsted P, Mallonee AB, Bergman NH, Koren S, Phillippy AM. Genome Biol. 2016 Jun 20;17(1):132. doi: 10.1186/s13059-016-0997-x](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-016-0997-x).

## Scikit-bio

[Scikit-bio](http://scikit-bio.org/) est une bibliothèque permettant de travailler avec des données biologiques en Python 3. On peut voir cette librairie comme un équivalent/complément à [biopython](https://biopython.org/).

Il existe plusieurs modules permettant :

* la gestion des formats fréquemment manipulés en bioinformatique via [skbio.io](http://scikit-bio.org/docs/latest/io.html)
* le stockage et la manipulation de séquences (nucléiques et protéiques) via [skbio.sequence](http://scikit-bio.org/docs/latest/sequence.html)
* la manipulation des alignements de séquences via [skbio.alignment](http://scikit-bio.org/docs/latest/alignment.html)
* ...

Ici, nous allons nous intéresser plus particulièrement à [skbio.tree](http://scikit-bio.org/docs/latest/tree.html). Ce module offre des fonctionnalités permettant de travailler avec des arbres, notamment des arbres phylogénétiques et des hiérarchies. Il permet de construire des arbres, de les parcourir de différentes manières, d'effectuer des comparaisons, d'extraire des sous-arbres, ...

## Jeu de données

Le jeu de données comprend 8 génomes de *Salmonella enterica subsp. enterica serovar Paratyphi* issus de la banque de données [RefSeq](https://www.ncbi.nlm.nih.gov/refseq/about/). Ils ont tous les 8, un niveau d'assemblage **Complete**.

## Import des librairies

In [1]:
import os
from skbio import DistanceMatrix
from skbio.tree import nj

## Parcours des fichiers contenant la distance de Mash entre chaque génome

Le sous-répertoire **data/data_mashDist** contient les fichiers de sortie de l'outil `mash dist` avec le calcul des distances Mash pour chaque paire de génomes. Voici un exemple de fichier de sortie pour un calcul de distance entre les génomes **GCF_000486725.2_ASM48672v2_genomic** et **GCF_000818115.1_ASM81811v1_genomic** ayant une distance de Mash de **0.00105797** :

```
Reference-ID	Query-ID	Mash-distance	P-value	Matching-hashes
GCF_000486725.2_ASM48672v2_genomic.fna.gz	GCF_000818115.1_ASM81811v1_genomic.fna.gz	0.00105797	0	957/1000
```

Dans notre cas, ce sont les colonnes **Reference-ID**, **Query-ID** et **Mash-distance** qui vont particulièrement nous intéresser.

In [2]:
# Récupération de l'ensemble des fichiers contenant les distances entre génomes
files = os.listdir('../data/mashDist')

# Initialisation de variables
dist = {}      # dictionnaire contenant l'ensemble des distances de Mash pour chaque paire de génomes
names = []     # liste contenant les noms des huit génomes

# Parcours de l'ensemble des fichiers contenant les distances entre génomes
for file in sorted(files):
    # Ouverture du fichier en lecture
    with open('../data/mashDist/'+file, "r") as distfile:
        # Parcours du fichier ligne par ligne
        for line in distfile:
            # Création d'une liste grâce au séparateur '\t'
            liste = line.split('\t')
            # Stockage des données dans un dictionnaire
            # Clé : tuple avec le nom des deux génomes
            # Valeur : distance de Mash entre ces deux génomes
            dist[(liste[0], liste[1])] = liste[2]
            # Ajout des noms de génomes dans une liste
            if liste[0] not in names:
                names.append(liste[0])
            if liste[1] not in names:
                names.append(liste[1])

# Classement par ordre ASCII des noms de génomes
names = sorted(names)

## Création d'une matrice de distance

La fonction `DistanceMatrix()` de la librairie [scikit-bio](http://scikit-bio.org/) permet de créer une matrice de distance à partir d'une liste de listes. Il est donc nécessaire de transformer les données stockées dans notre dictionnaire en une structure de type liste de listes.

In [3]:
# Initialisation de variable
matrix = []    # liste contenant l'ensemble des données de distance de Mash

# Parcours de l'ensemble des génomes
for nameA in names:
    # Initialisation d'une liste qui va contenir 
    # l'ensemble des distances pour ce génome
    dist_tmp = []
    # Parcours de l'ensemble des génomes
    for nameB in names:
        # Tuple avec les noms des génomes
        # Clé du dictionnaire contenant les distances
        key = (nameA, nameB)
        key2 = (nameB, nameA)
        # Récupération de la distance
        if key in dist:
            dist_tmp.append(float(dist[key]))
        elif key2 in dist:
            dist_tmp.append(float(dist[key2]))
        # Distance = 0 (génome contre lui-même)
        else:
            dist_tmp.append(0)
    # Ajout des distances pour ce génome à la matrice final
    matrix.append(dist_tmp)

# Création de la matrice avec scikit-bio
dm = DistanceMatrix(matrix, names)

## Création d'un arbre de Neighbor joining

La fonction `nj()` de la librairie `scikit-bio` permet de créer un arbre phylogénétique via la méthode de [Neighbor joining](https://fr.wikipedia.org/wiki/Neighbour_joining).

In [4]:
# Création de l'arbre
tree = nj(dm)

# Affichage de l'arbre
print(tree.ascii_art())

                    /-GCF 000011885.1 ASM1188v1 genomic.fna.gz
          /--------|
         |         |          /-GCF 000026565.1 ASM2656v1 genomic.fna.gz
         |          \--------|
         |                   |          /-GCF 000486725.2 ASM48672v2 genomic.fna.gz
         |                    \--------|
         |                             |          /-GCF 000018705.1 ASM1870v1 genomic.fna.gz
---------|                              \--------|
         |                                        \-GCF 000018385.1 ASM1838v1 genomic.fna.gz
         |
         |          /-GCF 000818115.1 ASM81811v1 genomic.fna.gz
         |---------|
         |          \-GCF 000818075.1 ASM81807v1 genomic.fna.gz
         |
          \-GCF 002386165.1 ASM238616v1 genomic.fna.gz


## Format Newick de l'arbre

In [5]:
newick_str = nj(dm, result_constructor = str)
print(newick_str)

((GCF_000011885.1_ASM1188v1_genomic.fna.gz:0.000049, (GCF_000026565.1_ASM2656v1_genomic.fna.gz:0.000017, (GCF_000486725.2_ASM48672v2_genomic.fna.gz:0.000319, (GCF_000018705.1_ASM1870v1_genomic.fna.gz:0.007044, GCF_000018385.1_ASM1838v1_genomic.fna.gz:0.007259):0.009483):0.000687):0.000007):0.000046, (GCF_000818115.1_ASM81811v1_genomic.fna.gz:0.000000, GCF_000818075.1_ASM81807v1_genomic.fna.gz:0.000047):0.000001, GCF_002386165.1_ASM238616v1_genomic.fna.gz:0.000023);


<br/>Pour aller plus loin avec la représentation sous forme d'arbre, cf. [http://scikit-bio.org/docs/0.2.1/tree.html](http://scikit-bio.org/docs/0.2.1/tree.html).